**🧠 Adaptive Personal Study Coach — Multi-Agent System **

A notebook that listens, learns, and reshapes your study patterns. A study planner that behaves less like a rigid timetable and more like a gentle compass — adjusting direction as your energy, mood, and habits ebb and glow. This system blends Gemini reasoning, rule-based agents, and lightweight storage into a coordinated trio:

Memory Agent — remembers what you did, wanted to do, and forgot to do
Planner Agent — takes a task and unmasks its smaller bones (subtasks)
Recommender Agent — reads your energy/mood levels and shapes a schedule
Controller — orchestrates everything, ensuring responses don’t trip over each other

We’ll build all of this in calm, deliberate stages — each wrapped in its own section, like chapters in a small technical fable.

DATA FLOW DIAGRAM (Conceptual)
flowchart TD

    U[User: Task + Mood + Energy] --> C[Controller]

    C --> P[Planner Agent<br>Gemini decomposition]
    C --> R[Recommender Agent<br>Mood-Energy reasoning]

    P --> C
    R --> C

    C --> M[Memory Agent<br>TinyDB storage]

    C --> O[Final Optimized Study Plan]

    M --> H[History / Logs]
This mermaid diagram gently maps out the pipeline of thought from user → agents → memory.

In [1]:
!pip install --quiet tinydb python-dateutil


🔧 SECTION 1 — Imports & Utilities


In [2]:
from tinydb import TinyDB, Query
from datetime import datetime
from dateutil import parser
import uuid
import os
import requests
import json


A few helper utilities (timestamps, safe prints, etc.) will appear later as needed — no clutter yet.

🔐 SECTION 2 — Gemini API Key Authentication

Kaggle likes secrets tucked away safely.
This cell gently retrieves the API key and sets it into the environment.

In [3]:
# ============================================================
# 🔐 Gemini API Key Authentication (Kaggle Secrets)
# ============================================================

from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please add 'GOOGLE_API_KEY' to Kaggle Secrets. Details: {e}"
    )


✅ Gemini API key setup complete.


🌩 SECTION 3 — Gemini API Wrapper

A small, quiet function that handles all communication with the Gemini model.

In [4]:
API_KEY = os.environ.get("GOOGLE_API_KEY")
MODEL = "gemini-2.5-flash"
URL = f"https://generativelanguage.googleapis.com/v1beta/models/{MODEL}:generateContent?key={API_KEY}"

def ask_gemini(prompt: str) -> str:
    """
    Sends a prompt to Gemini and returns the generated text.
    """
    body = {"contents": [{"parts": [{"text": prompt}]}]}
    
    response = requests.post(URL, json=body)
    data = response.json()
    
    try:
        return data["candidates"][0]["content"]["parts"][0]["text"]
    except:
        return f"[Gemini Error] {data}"


🎒 SECTION 4 — TinyDB Memory Layer

A memory not as fragile as RAM, not as heavy as SQL — just right for our quiet little agent.

In [5]:
db = TinyDB("memory.json")
Memory = Query()

def save_memory(entry):
    entry["id"] = str(uuid.uuid4())
    entry["timestamp"] = datetime.now().isoformat()
    db.insert(entry)

def get_all_memory():
    return db.all()


🗂 SECTION 5 — Memory Agent

A soft archivist that stores:

What tasks you planned

What you finished

What overwhelmed you

What mood/energy you were in

Think of it as a journal written in tiny, structured whispers.

In [6]:
db = TinyDB("memory.json")
Memory = Query()

def save_memory(entry):
    entry["id"] = str(uuid.uuid4())
    entry["timestamp"] = datetime.now().isoformat()
    db.insert(entry)

def get_all_memory():
    return db.all()


In [7]:
class MemoryAgent:
    def __init__(self, db):
        self.db = db

    def record_task(self, task, status, mood=None, energy=None):
        entry = {
            "type": "task_record",
            "task": task,
            "status": status,
            "mood": mood,
            "energy": energy,
        }
        save_memory(entry)

    def record_feedback(self, feedback):
        entry = {"type": "feedback", "feedback": feedback}
        save_memory(entry)

    def fetch_history(self):
        return get_all_memory()


In [8]:
memory_agent = MemoryAgent(db)
memory_agent.record_task("Study Operating Systems", "completed", mood="calm", energy="medium")


This deposits a small story into your “memory.json” vault.

🧩 SECTION 6 — Planner Agent

The planner is a sculptor — it takes a rough task like:

“Prepare for Machine Learning exam”

… and chips it into finer, workable pieces:

Collect syllabus

Gather notes

Solve PYQs

Run revision loops

It uses Gemini for decomposition so that tasks become smooth and chewable.

In [9]:
class PlannerAgent:
    def __init__(self):
        pass

    def decompose(self, task: str):
        prompt = (
            "Break the following study task into 5–8 actionable subtasks. "
            "Respond only with a numbered list.\n\n"
            f"Task: {task}"
        )
        output = ask_gemini(prompt)
        return output


In [10]:
planner = PlannerAgent()
subtasks = planner.decompose("Revise Data Structures")
print(subtasks)


1.  Review course syllabus and previous notes to identify all topics covered.
2.  Create a quick reference guide (flashcards or summary sheet) for each data structure, including definitions, properties, and common operations.
3.  Analyze and memorize the time and space complexity (Big O notation) for key operations on each data structure.
4.  Review pseudo-code or actual code implementations for core operations and algorithms (e.g., traversals, insertions, deletions).
5.  Solve a variety of practice problems that require applying different data structures to real-world scenarios.
6.  Identify specific data structures or algorithms you struggled with during practice.
7.  Deep-dive into those identified weak areas through additional examples, tutorials, or focused practice.
8.  Take a self-assessment or work through a mock exam to test overall understanding and recall.


You’ll receive a neatly chiseled list of steps.

⚖️ SECTION 7 — Recommender Agent

This one listens to your mood and energy, like a quiet physiologist.

Low energy? → short, soft tasks

High motivation? → deep work

Stressed? → review-oriented schedule

Bored? → variety injection

In [11]:
class RecommenderAgent:
    def __init__(self):
        pass

    def recommend(self, subtasks: str, mood: str, energy: str):
        prompt = (
            f"You are an adaptive study recommender.\n"
            f"Given the user's mood: {mood} and energy: {energy},\n"
            f"reorder or enhance the following subtasks to create an optimal study sequence.\n\n"
            f"Subtasks:\n{subtasks}"
        )
        return ask_gemini(prompt)


In [12]:
recommender = RecommenderAgent()
rec = recommender.recommend(
    subtasks,
    mood="tired",
    energy="low"
)
print(rec)


Given your mood (tired) and energy (low), the key is to prioritize tasks that are less mentally demanding, offer quick wins, and can be done in short, focused bursts. Avoid tasks that require sustained deep analytical thinking or can lead to frustration.

Here's an optimized study sequence, enhanced for your current state:

---

**Optimal Study Sequence for Tired & Low Energy:**

**Phase 1: Gentle Warm-up & Overview (Minimal Cognitive Load)**

1.  **Review course syllabus and previous notes to identify all topics covered.**
    *   **Enhancement:** Don't re-read everything in detail. Instead, **skim through topic headings** to refresh your memory of the scope. Focus on creating a mental checklist of what you *need* to cover, not on mastering it now. This is about orientation and activating prior knowledge.
    *   *(Goal: 5-10 minutes)*

2.  **Briefly review existing quick reference guides or flashcards.**
    *   **Enhancement:** If you already have flashcards or summary sheets, simpl

🎛 SECTION 8 — Controller Agent (the conductor)

This ties everything together.
A maestro waving calmly while the Memory, Planner, and Recommender play their notes.

In [13]:
class Controller:
    def __init__(self, memory_agent, planner_agent, recommender_agent):
        self.memory = memory_agent
        self.planner = planner_agent
        self.recommender = recommender_agent

    def process_task(self, task, mood, energy):
        subtasks = self.planner.decompose(task)
        plan = self.recommender.recommend(subtasks, mood, energy)

        self.memory.record_task(task, "planned", mood, energy)

        return {
            "subtasks": subtasks,
            "final_plan": plan,
            "history": self.memory.fetch_history()
        }


In [14]:
controller = Controller(memory_agent, planner, recommender)

result = controller.process_task(
    task="Prepare for DBMS exam",
    mood="neutral",
    energy="medium"
)

result


{'subtasks': '1.  **Consolidate All Study Materials:** Gather lecture notes, textbook chapters, assignments, and past quizzes/exams.\n2.  **Outline Key DBMS Topics:** Create a comprehensive list of all major concepts to be covered (e.g., ER models, Relational Algebra, SQL, Normalization, Transactions, Indexing, Concurrency, Recovery).\n3.  **Review Core Concepts and Theory:** Systematically go through each topic, focusing on understanding definitions, principles, and algorithms (e.g., ACID properties, B-trees, join types).\n4.  **Practice SQL Querying and DDL/DML:** Solve a variety of SQL problems, including SELECT, INSERT, UPDATE, DELETE, CREATE TABLE, JOINs, subqueries, and aggregate functions.\n5.  **Work Through Practical Problems:** Practice ER diagramming, normalization (1NF to BCNF/4NF), transaction schedule analysis, and index cost estimation.\n6.  **Complete Practice Exams/Questions:** Work through past exam papers, textbook practice problems, or online quizzes under timed con

🧪 SECTION 9 — First System Test (Dry Run)

A gentle warm-up run to make sure all the gears mesh.

In [15]:
# Quick system test
controller = Controller(memory_agent, planner, recommender)

test_output = controller.process_task(
    task="Revise Computer Networks",
    mood="relaxed",
    energy="high"
)

test_output


{'subtasks': '1.  Review the course syllabus and learning objectives to identify all covered topics.\n2.  Skim through all lecture notes and textbook chapters to get an overview and identify familiar versus challenging areas.\n3.  Deeply study specific challenging topics, consulting textbooks, online resources, and supplementary materials as needed.\n4.  Work through practice problems, past exam questions, or lab exercises related to core concepts.\n5.  Create summary notes, flashcards, or mind maps for key protocols, architectures (e.g., OSI, TCP/IP), and algorithms.\n6.  Explain complex concepts (e.g., routing protocols, congestion control) aloud or to a study partner to solidify understanding.\n7.  Review network topologies and protocol interactions using diagrams and example scenarios.\n8.  Conduct a final self-assessment or timed practice session to gauge readiness.',
 'final_plan': "This is an excellent state for tackling substantial learning! Your high energy allows for deep eng

📚 SECTION 10 — Memory Explorer Cell

A tiny window into the notebook’s inner diary.
Useful for debugging, evaluation, or curiosity.

In [16]:
# Inspect what the system remembers so far
memory_agent.fetch_history()


[{'type': 'task_record',
  'task': 'Study Operating Systems',
  'status': 'completed',
  'mood': 'calm',
  'energy': 'medium',
  'id': '2e853ecb-3a63-4dcb-9ad1-9c304fbc7c05',
  'timestamp': '2025-11-30T12:49:31.758573'},
 {'type': 'task_record',
  'task': 'Prepare for DBMS exam',
  'status': 'planned',
  'mood': 'neutral',
  'energy': 'medium',
  'id': '6094a9ce-f86a-4403-b013-a825618dac08',
  'timestamp': '2025-11-30T12:50:15.132126'},
 {'type': 'task_record',
  'task': 'Revise Computer Networks',
  'status': 'planned',
  'mood': 'relaxed',
  'energy': 'high',
  'id': '766537a1-a34d-49df-b978-dbf08967c145',
  'timestamp': '2025-11-30T12:50:34.131978'}]

🎛 SECTION 11 — Interactive Study Prompt (Mini UI)

A soft, terminal-style interface inside the notebook.
Lightweight, no libraries required.

This turns your notebook into a calm conversational partner.

🪞 SECTION 12 — Feedback Loop 

Improves the system over time by storing your reflections.

In [17]:
import gradio as gr

def interact_ui(task, mood, energy):
    result = controller.process_task(task, mood, energy)
    
    return (
        result["subtasks"],
        result["final_plan"],
        "Memory updated successfully ✔️"
    )

ui = gr.Interface(
    fn=interact_ui,
    inputs=[
        gr.Textbox(label="Enter your study task"),
        gr.Dropdown(["calm", "stressed", "neutral"], label="Your current mood"),
        gr.Dropdown(["low", "medium", "high"], label="Your energy level"),
    ],
    outputs=[
        gr.Textbox(label="🧩 Subtasks"),
        gr.Textbox(label="🎯 Personalized Plan"),
        gr.Textbox(label="🗂 Memory Status")
    ],
    title="📌 Adaptive Study Planner",
    description="Gemini-powered multi-agent study planner"
)

ui.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://590501965511084b03.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🧹 SECTION 13 — Memory Cleaning Utilities (

Admins sometimes want to wipe memory before submitting or testing.

In [18]:
def clear_memory(confirm=False):
    if confirm:
        db.truncate()
        print("🧼 Memory wiped clean.")
    else:
        print("Set confirm=True to wipe memory.")

# Example: clear_memory(confirm=True)


🌙 14 — COMPLETION BANNER

In [19]:
print("✨ Study Coach Notebook fully initialized.")


✨ Study Coach Notebook fully initialized.
